In [11]:
import sys; sys.path.insert(0, '../')
from PlexSim.plexsim.models import *
n = 256
g = nx.grid_2d_graph(n, n, periodic = 1)

settings = dict(graph = g,\
                updateType = 'sync',\
                )


models = dict(
#     Potts = Potts(agentStates = [0, 1, 2],\
#           **settings),\
#     Ising = Ising(agentStates = np.arange(0, 4).tolist(),\
#           **settings),\
#     RBN = RBN(**settings),\
#     SIRS = SIRS(\
#         mu = 0.05, \
#         kappa = .001,\
#         beta = .5, \
#         **settings),\
#     CCA(\
#         agentStates = np.arange(0,4).tolist(),\
#         threshold = .001, **settings),
#     Percolation(p = .2,\
#                **settings),\
   Bornholdt = Bornholdt(t = .5,\
              alpha = 4,\
              sampleSize = .25,\
              **settings)
)

# models.get("Bornholdt").sampleSize = 1
if m := models.get("SIRS"):
    m.states = 0
    m.states[m.sampleNodes(1)[0,0]] = 1
T = 1000

import time
start = time.time()
results = np.zeros((len(models), T, g.number_of_nodes()))
for idx, m in enumerate(models.values()):
    results[idx] = m.simulate(T)
results = results.reshape(-1, T, n, n)
print(time.time() - start)

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable as mal
%matplotlib widget
ts = IntSlider(min = 0, max = T - 1, value = 0)
fig, ax = plt.subplots(1, 2, figsize = (5,5))

fig.subplots_adjust(wspace = .12, hspace = .0001)
#setup figure
hs  = []
pad = .01
size = '5%'
for idx, ( (modelName, m), axi) in enumerate(zip(models.items(), ax.flat)):
    div = mal(axi)
    cbr = div.append_axes('bottom', size = size, pad = pad)
#     div = mal(cbr)
#     cbr = div.append_axes('right', size = size, pad = pad)
    AS = m.agentStates
    NS = len(AS) 
    cmap = mpl.colors.LinearSegmentedColormap.from_list(\
          None, plt.cm.viridis(np.linspace(0, 255, NS,\
                                          dtype = int)), NS)
    h = axi.imshow(results[idx, 0], cmap = cmap, vmin = AS[0],\
                  vmax = AS[-1])
    axi.set_title(modelName)
    axi.axis("off")
    tmp = fig.colorbar(h, cax = cbr, ticks = AS,\
                       orientation = 'horizontal')
    if modelName == 'SIRS':
        tmp.set_ticklabels("Sus. Inf. Rec.".split())
    hs.append(h)
mainax = fig.add_subplot(111, frameon = 0,\
                        xticks = [], yticks = [])
mainax.set_xlabel(f'T = 0')
@interact(t = ts)
def update(t):
    for idx, h in enumerate(hs):
        h.set_data(results[idx, t])
    mainax.set_xlabel(f'T = {t}')
    fig.canvas.flush_events()
    fig.canvas.draw()
fig.show()

17.218539476394653


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…

In [12]:
import ipywidgets, asyncio
m = models.get("Bornholdt")
print(m.sampleSize)
m.sampleNodes(10).base

16384


array([[ 9236, 11569, 31863, ..., 23432, 54270, 24593],
       [ 9236, 11569, 31863, ..., 23432, 54270, 24593],
       [ 9236, 11569, 31863, ..., 23432, 54270, 24593],
       ...,
       [58954,  5112, 51408, ...,  8393, 58120,  5181],
       [58954,  5112, 51408, ...,  8393, 58120,  5181],
       [58954,  5112, 51408, ...,  8393, 58120,  5181]])

In [3]:
out

NameError: name 'out' is not defined

In [ ]:
from matplotlib.animation import FuncAnimation as fan
#an = fan(fig, update, frames = np.arange(0, int(T*1), 10), interval = 100,\

#         blit = True) 
#an.save('PlexSim_banner.mp4', writer = 'ffmpeg')